In [1]:
import numpy as np
import pandas as pd
import math
from numpy import delete
from tqdm import tqdm
import random

In [3]:
movies

,m_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995)
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...
9,10,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...


In [2]:
users = pd.read_csv('u.user', sep='|', names=['u_id','age','sex', 'occupation','zip_code'], encoding='latin-1')
ratings = pd.read_csv(r'u.data',sep='\t', names=['u_id','m_id','rating','unix_timestamp'],encoding='latin-1')
movies = pd.read_csv('u.item',sep='|',names=['m_id','title','release_date','video_release_date','imdb_url'],usecols=range(5),encoding='latin-1')
movie_ratings = pd.merge(movies, ratings)
alls = pd.merge(movie_ratings, users)
print(alls)

       m_id                                   title release_date  \
0         1                        Toy Story (1995)  01-Jan-1995   
1         4                       Get Shorty (1995)  01-Jan-1995   
2         5                          Copycat (1995)  01-Jan-1995   
3         7                   Twelve Monkeys (1995)  01-Jan-1995   
4         8                             Babe (1995)  01-Jan-1995   
5         9                 Dead Man Walking (1995)  01-Jan-1995   
6        11                    Seven (Se7en) (1995)  01-Jan-1995   
7        12              Usual Suspects, The (1995)  14-Aug-1995   
8        15               Mr. Holland's Opus (1995)  29-Jan-1996   
9        17              From Dusk Till Dawn (1996)  05-Feb-1996   
10       19                   Antonia's Line (1995)  01-Jan-1995   
11       21           Muppet Treasure Island (1996)  16-Feb-1996   
12       22                       Braveheart (1995)  16-Feb-1996   
13       23                      Taxi Driver (19

[100000 rows x 12 columns]


In [4]:
print('设置一个矩阵,横向是用户ID，纵向是电影的ID')
user_movie = np.zeros([943, 1682])
for i in range(len(ratings.values)):
    user_movie[ratings.values[i][0]-1][ratings.values[i][1]-1]=ratings.values[i][2]
    
print('user_movie:',user_movie[:2,:])

设置一个矩阵,横向是用户ID，纵向是电影的ID
user_movie: [[ 5.  3.  4. ...,  0.  0.  0.]
 [ 4.  0.  0. ...,  0.  0.  0.]]


In [5]:
from numpy import *
eps = 1.0e-6
def cosSim(inA, inB):
    denom = linalg.norm(inA)*linalg.norm(inB)
    return float(dot(inA,inB.T))/denom+eps

In [6]:
def svd_(new,r=2,rank=1):
    U, S, VT = linalg.svd(user_movie.T)
    V = VT.T
    Sigma = diag(S)
    Ur = U[:,:r]
    Sr = Sigma[:r,:r]
    Vr = V[:,:r]
    print('Ur=',Ur.shape,'Vr=',Vr.shape)
    newresult = dot(dot(new,Ur),linalg.inv(Sr))
    print('newresult=',newresult)
    
    maxv = 0     # 最大的余弦值
    maxi = 0     # 最大值得下标
    indx = 0
    dic = {}
    for vi in Vr:
        temp = cosSim(newresult, vi)
        if temp > maxv:
            maxv = temp
            maxi = indx
        indx +=1
        dic.update({temp:indx})
    ress = sorted(dic.items(), key=lambda item:item[0])[-rank:]
    print(ress)
    return ress[0],ress[1]

In [7]:
test = pd.read_csv(r'u1.test',sep='\t', names=['u_id','m_id','rating','unix_timestamp'], encoding='latin-1')
test_list = np.zeros([943, 1682])
for i in range(len(test.values)):
    test_list[test.values[i][0]-1][test.values[i][1]-1] = test.values[i][2]

In [8]:
resa,resb = svd_(test_list[22], rank=2)       # 开始预测

Ur= (1682, 2) Vr= (943, 2)
newresult= [-0.01823787 -0.00884465]
[(0.99999921907629408, 307), (0.99999980081101902, 753)]


In [9]:
box = []
def pre(resa):
    global box
    for i in range(len(user_movie[resa[1]])):
        if user_movie[resa[1]][i] != 0:
            if test_list[22][i] == 0:
                box+=[movies.loc[i]['title']]
    return box
boxx = pre(resa)

In [10]:
print(len(boxx))
print(boxx)

356


['Get Shorty (1995)', 'Copycat (1995)', 'Twelve Monkeys (1995)', 'Babe (1995)', 'Dead Man Walking (1995)', 'Seven (Se7en) (1995)', 'Usual Suspects, The (1995)', "Mr. Holland's Opus (1995)", 'From Dusk Till Dawn (1996)', "Antonia's Line (1995)", 'Muppet Treasure Island (1996)', 'Braveheart (1995)', 'Taxi Driver (1976)', 'Rumble in the Bronx (1995)', 'Birdcage, The (1996)', 'Belle de jour (1967)', 'Crimson Tide (1995)', 'Clerks (1994)', 'Dolores Claiborne (1994)', 'Eat Drink Man Woman (1994)', 'Ed Wood (1994)', 'Hoop Dreams (1994)', 'I.Q. (1994)', 'Outbreak (1995)', 'Professional, The (1994)', 'Pulp Fiction (1994)', 'Quiz Show (1994)', 'Three Colors: Red (1994)', 'Three Colors: Blue (1993)', 'Three Colors: White (1994)', 'Shawshank Redemption, The (1994)', "What's Eating Gilbert Grape (1993)", 'While You Were Sleeping (1995)', 'Crow, The (1994)', 'Forrest Gump (1994)', 'Mask, The (1994)', 'Maverick (1994)', 'Faster Pussycat! Kill! Kill! (1965)', 'Firm, The (1993)', 'Fugitive, The (1993)'